# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
</div>
****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
df = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

pd.set_option('display.max_columns', 200)
df[['race','call']].head() # main variables are 'race' and 'call'

,race,call
0,w,0.0
1,w,0.0
2,b,0.0
3,b,0.0
4,w,0.0


In [2]:
df['call'] = df['call'].astype(int) # change variable 'call' to be an integer 
df_call = df[['race','call']]
df_call.head()


,race,call
0,w,0
1,w,0
2,b,0
3,b,0
4,w,0


In [3]:
Black_called = sum(df_call[df_call.race=='b'].call)
White_called = sum(df_call[df_call.race=='w'].call)
Black_total = len(df_call[df_call.race=='b'].call)
White_total = len(df_call[df_call.race=='b'].call)

Black_pct = Black_called/Black_total
White_pct = White_called/White_total

print('Total black sample = ' + str(Black_total) + ', ' + str(Black_called) + ' called (prob = ' + "{0:.4f}".format(Black_pct) + ').')
print('Total white sample = ' + str(White_total) + ', ' + str(White_called) + ' called (prob = ' + "{0:.4f}".format(White_pct) + ').')

Total black sample = 2435, 157 called (prob = 0.0645).
Total white sample = 2435, 235 called (prob = 0.0965).


#### Observations
* Overall, being white seems to have higher probability of being called (9.7%) than being black (6.45%)

#### [Question 1] What test is appropriate for this problem? Does CLT apply?
* We should use two-sample two-sided t-test on probability of being called (mean of variable "call").
* The two samples are black (race == b) and white (race == white). 
* Sample size is rather large, 2435 for both black and white, so we can apply CLT to the test.  

#### [Question 2]  What are the null and alternate hypotheses?
* Null hypothesis is that the probability of being called is the same between white and black.
* Alternate hypothesis is that the probability of being called is different between white and black. 


#### [Question 3] Compute margin of error, confidence interval, and p-value.

In [7]:
print("Null hypothesis: prob(call|being white) - prob(call|being black) = 0")
print("---------")

diff_mean = White_pct - Black_pct

print("difference in mean = " + "{0:.4f}".format(diff_mean))

var_Black_sample = (Black_pct*(1-Black_pct))*(Black_total)/(Black_total-1)
var_White_sample = White_pct*(1-White_pct)*(Black_total)/(Black_total-1)

sigma = np.sqrt( var_Black_sample/Black_total + var_White_sample/White_total ) 
print("estimated std of sampling diff in mean = " + "{0:.4f}".format(sigma))

t_critical = stats.t.ppf( q = 0.975, df = Black_total-1 )

margin_of_error = sigma*t_critical
print("margin of error = " + "{0:.4f}".format(margin_of_error))

CI = (diff_mean - margin_of_error, diff_mean + margin_of_error)
print("confidence interval (95% CI) : [" + "{0:.4f}".format(CI[0]) + ', ' + "{0:.4f}".format(CI[1]) + ']')

t_statistic = diff_mean/sigma
print('t_statistic = ' + str(t_statistic))

print("---------")
print('Python built-in t-test: ')
print("---------")
black_call = np.array(df[df['race']=='b']['call'])
white_call = np.array(df[df['race']=='w']['call'])
stats.ttest_ind(white_call,black_call)

Null hypothesis: prob(call|being white) - prob(call|being black) = 0
---------
difference in mean = 0.0320
estimated std of sampling diff in mean = 0.0078
margin of error = 0.0153
confidence interval (95% CI) : [0.0168, 0.0473]
t_statistic = 4.11470526672
---------
Python built-in t-test: 
---------


Ttest_indResult(statistic=4.114705266723095, pvalue=3.9408025140695284e-05)

* pvalue is extremely low and we reject the null hypothesis that probability of being called is the same between black and white
#### [Question 4] Write a story describing the statistical significance in the context or the original problem.

The experiment is to randomly assign identical resumes to black-sounding or white-sounding names and see the response of employers. Having a white-sounding name in the resume increases probability of being called by 3.2% than having a black-sounding name. Other things constant, there seems to be racial discrimination in the US job market.     

#### [Question 5] Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

No, this experiment controls for all other potentially important variables constant and see if race/name can predict the probability of being called. To be able to see what other factors are important, we can also use other variables and perform logistic regression. 